# La idea del proyecto final va a ser conectarse a la API de Spotify y teniendo en cuenta el top 50 global de varios países y otras playlist aleatorias, ver cual de los "New releases" van a ser populares

In [156]:
#Importación de todas las librerias necesarias

import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
from pandas import json_normalize
from datetime import datetime
import os


#API

import requests
import json

### Consumo de datos - conexión a la API

In [157]:
#Requerir token a la API

url_access = "https://accounts.spotify.com/api/token"
client_id = "" # Aquí irían las contraseñas que se obtienen de la API de Spotify cuando uno ingresa con su cuenta
client_secret = "" # Aquí irían las contraseñas que se obtienen de la API de Spotify cuando uno ingresa con su cuenta

# Configurar los headers
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

# Configurar los datos del cuerpo de la solicitud
data = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret
}

# Hacer la solicitud POST
response = requests.post(url_access, headers=headers, data=data)

# Imprimir la respuesta
respuesta = response.json()
access_token = respuesta.get("access_token")
access_token

'BQAwTCfb8JO4pc9FawJZw7ekKs10x0JwbWX7KjtkoTrzKH3yPU1jhBjp22kSOadPy28cqhUER6yV9wXMsDow2nW0mMWaukiKYCuuM8otMX-IUPJqohk'

In [158]:
#Busco la playlist TOP-50 GLOBAL

id_top50 = "37i9dQZEVXbMDoHDwVN2tF"

# URL del endpoint de Spotify para obtener la playlist
url_top50 = f"https://api.spotify.com/v1/playlists/{id_top50}/tracks"

# Configurar los headers
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Hacer la solicitud GET
response = requests.get(url_top50, headers=headers)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Obtener la respuesta JSON
    response_json = response.json()
    # Imprimir la respuesta JSON
    #print(response_json)
else:
    print(f"Error: {response.status_code}")
    print(response.json())

### Luego de ciertos analisis se llego a la conlcusion que serian pocos datos usar unicamente el top50 global, por lo que se agregaron mas playlists al analisis

In [159]:
def get_playlist_tracks(playlist_id, access_token):
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data for playlist {playlist_id}: {response.status_code}")
        return None

# IDs de las playlists, se eligieron aleatoriamente, buscando repartir entre distintos paises de distintos continentes
playlist_ids = [
    '37i9dQZEVXbMMy2roB9myp',  # Top 50 Argentina
    '37i9dQZEVXbLRQDuF5jeBp',  # Top 50 USA
    '37i9dQZEVXbKuaTI1Z1Afx',  # Top 50 UK
    '37i9dQZEVXbMXbN3EUUhlg',  # Top 50 Brasil
    '37i9dQZEVXbNFJfN1Vw8d9',  # Top 50 España
    '37i9dQZEVXbKXQ4mDTEBXq',  # Top 50 Japón
    '37i9dQZEVXbNxXF4SkHj9F',  # Top 50 Corea
    '37i9dQZEVXbO3qyFxbkOE1',  # Top 50 Mexico
    '37i9dQZEVXbKj23U1GF4IR',  # Top 50 Canada
    '37i9dQZEVXbJPcfkRz0wJ0',  # Top 50 Australia
    '37i9dQZEVXbIPWwFssbupI',  # Top 50 Francia
    '6GxG78ydt6XoKjXJtBI9rS',  # TwentyOnePilots
    '37i9dQZEVXcGhk7S8X2AMt',  # Discover Daily
    '37i9dQZF1DZ06evO4uy2je',  # This is Soda Stereo
    '37i9dQZF1DWSo7PX7dbgH8',  # Rock Nacional: 100% clásicos
    '37i9dQZF1Fa1IIVtEpGUcU',  # Your top songs 2023
    '37i9dQZF1E38wC9SrBrgfu',  # Daily Mix 3
    '63VME6kq1xgALhi0H6gEIg',  # Chapelcolab (propia)
    '5llgBd4sY9rmMNf0U6zAqu',  # Rock Nacional (propia)
    '37i9dQZF1DWSV3Tk4GO2fq',  # All Out 50s
    '37i9dQZF1DXaKIA8E7WcJj',  # All Out 60s
    '37i9dQZF1DWTJ7xPn4vNaz',  # All Out 70s
    '37i9dQZF1DX4UtSsGT1Sbe',  # All Out 80s
    '37i9dQZF1DXbTxeAdrVG2l',  # All Out 90s
    '37i9dQZF1DX4o1oenSJRJd',  # All Out 2000s
    '2oD3XGPZCoxQgqhbX5N8p3',  # . (propia)
]

df_all_tracks = pd.DataFrame()

for playlist_id in playlist_ids:
    playlist_data = get_playlist_tracks(playlist_id, access_token)
    df_each_track = pd.DataFrame(playlist_data['items'])
    df_all_tracks = pd.concat([df_all_tracks, df_each_track], ignore_index=True)

df_all_tracks

,added_at,added_by,is_local,primary_color,track,video_thumbnail
0,2024-09-13T12:55:41Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,"{'preview_url': None, 'available_markets': ['A...",{'url': None}
1,2024-09-13T12:55:41Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,{'preview_url': 'https://p.scdn.co/mp3-preview...,{'url': None}
2,2024-09-13T12:55:41Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,{'preview_url': 'https://p.scdn.co/mp3-preview...,{'url': None}
3,2024-09-13T12:55:41Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,{'preview_url': 'https://p.scdn.co/mp3-preview...,{'url': None}
4,2024-09-13T12:55:41Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,{'preview_url': 'https://p.scdn.co/mp3-preview...,{'url': None}
...,...,...,...,...,...,...
1733,2020-02-28T17:57:59Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,{'preview_url': 'https://p.scdn.co/mp3-preview...,{'url': None}
1734,2020-02-29T22:51:54Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,{'preview_url': 'https://p.scdn.co/mp3-preview...,{'url': None}
1735,2020-03-03T18:11:24Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,{'preview_url': 'https://p.scdn.co/mp3-preview...,{'url': None}
1736,2020-03-03T18:15:36Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,{'preview_url': 'https://p.scdn.co/mp3-preview...,{'url': None}


In [160]:
#Esta información corresponde al top50 global
#La información que se necesita esta bajo el atributo "tracks" y luego dentro de "items"
df_50Global = pd.DataFrame(response_json['items'])
df_50Global

,added_at,added_by,is_local,primary_color,track,video_thumbnail
0,2024-09-13T12:55:47Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,"{'preview_url': None, 'available_markets': ['A...",{'url': None}
1,2024-09-13T12:55:47Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,"{'preview_url': None, 'available_markets': ['A...",{'url': None}
2,2024-09-13T12:55:47Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,"{'preview_url': None, 'available_markets': ['A...",{'url': None}
3,2024-09-13T12:55:47Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,"{'preview_url': None, 'available_markets': ['A...",{'url': None}
4,2024-09-13T12:55:47Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,{'preview_url': 'https://p.scdn.co/mp3-preview...,{'url': None}
5,2024-09-13T12:55:47Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,{'preview_url': 'https://p.scdn.co/mp3-preview...,{'url': None}
6,2024-09-13T12:55:47Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,"{'preview_url': None, 'available_markets': ['A...",{'url': None}
7,2024-09-13T12:55:47Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,"{'preview_url': None, 'available_markets': ['A...",{'url': None}
8,2024-09-13T12:55:47Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,"{'preview_url': None, 'available_markets': ['A...",{'url': None}
9,2024-09-13T12:55:47Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,{'preview_url': 'https://p.scdn.co/mp3-preview...,{'url': None}


In [161]:
#Concateno ambos dataframe bajo el nombre df_tracks
df_tracks = pd.concat([df_all_tracks, df_50Global])
df_tracks

,added_at,added_by,is_local,primary_color,track,video_thumbnail
0,2024-09-13T12:55:41Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,"{'preview_url': None, 'available_markets': ['A...",{'url': None}
1,2024-09-13T12:55:41Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,{'preview_url': 'https://p.scdn.co/mp3-preview...,{'url': None}
2,2024-09-13T12:55:41Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,{'preview_url': 'https://p.scdn.co/mp3-preview...,{'url': None}
3,2024-09-13T12:55:41Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,{'preview_url': 'https://p.scdn.co/mp3-preview...,{'url': None}
4,2024-09-13T12:55:41Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,{'preview_url': 'https://p.scdn.co/mp3-preview...,{'url': None}
...,...,...,...,...,...,...
45,2024-09-13T12:55:47Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,"{'preview_url': None, 'available_markets': ['A...",{'url': None}
46,2024-09-13T12:55:47Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,{'preview_url': 'https://p.scdn.co/mp3-preview...,{'url': None}
47,2024-09-13T12:55:47Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,"{'preview_url': None, 'available_markets': ['A...",{'url': None}
48,2024-09-13T12:55:47Z,{'external_urls': {'spotify': 'https://open.sp...,False,None,{'preview_url': 'https://p.scdn.co/mp3-preview...,{'url': None}


In [162]:
#Las playlist de spotify se suelen actualizar al mismo tiempo
fecha_playlist = df_tracks['added_at'].iloc[0]
fecha_corta = fecha_playlist[:10]
print(f"Fecha actualización playlist: {fecha_corta}")

Fecha actualización playlist: 2024-09-13


In [163]:
df_items_expanded = pd.json_normalize(df_tracks['track'])
df_items_expanded

,preview_url,available_markets,explicit,type,episode,track,artists,disc_number,track_number,duration_ms,...,album.name,album.release_date,album.release_date_precision,album.uri,album.artists,album.external_urls.spotify,album.total_tracks,external_ids.isrc,external_urls.spotify,external_ids.spotify
0,None,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,[{'external_urls': {'spotify': 'https://open.s...,1.0,1.0,195824.0,...,Si Antes Te Hubiera Conocido,2024-06-21,day,spotify:album:5ylbxH7EqpsmHZCRuiYewS,[{'external_urls': {'spotify': 'https://open.s...,https://open.spotify.com/album/5ylbxH7EqpsmHZC...,1.0,USUG12400968,https://open.spotify.com/track/6WatFBLVB0x077x...,NaN
1,https://p.scdn.co/mp3-preview/fe8056d15a777894...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,[{'external_urls': {'spotify': 'https://open.s...,1.0,1.0,198495.0,...,Hoy,2024-05-22,day,spotify:album:4ENi7P1IV01E2jzbHZeKxn,[{'external_urls': {'spotify': 'https://open.s...,https://open.spotify.com/album/4ENi7P1IV01E2jz...,1.0,BK4DA2414210,https://open.spotify.com/track/0SoBfYjQHvL6cuW...,NaN
2,https://p.scdn.co/mp3-preview/da1767fc90650adf...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,[{'external_urls': {'spotify': 'https://open.s...,1.0,1.0,144148.0,...,DOCTOR,2024-08-29,day,spotify:album:3Z9VnhuHM0BmsNUVPWMd7y,[{'external_urls': {'spotify': 'https://open.s...,https://open.spotify.com/album/3Z9VnhuHM0BmsNU...,1.0,USSD12400465,https://open.spotify.com/track/5DSMm0BZXlSvqAr...,NaN
3,https://p.scdn.co/mp3-preview/481bfc885494bc41...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,[{'external_urls': {'spotify': 'https://open.s...,1.0,5.0,165054.0,...,GOTTI A,2024-07-04,day,spotify:album:3s6jmSUxBLRxzKdevnFmJS,[{'external_urls': {'spotify': 'https://open.s...,https://open.spotify.com/album/3s6jmSUxBLRxzKd...,14.0,USWL12401900,https://open.spotify.com/track/0c82YIGpQ5dCaXp...,NaN
4,https://p.scdn.co/mp3-preview/54bba76c1b34f181...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,[{'external_urls': {'spotify': 'https://open.s...,1.0,1.0,104566.0,...,La Lógica del Escorpión,2024-09-12,day,spotify:album:0k2JGzYRZQC3s3ZNTcyHoA,[{'external_urls': {'spotify': 'https://open.s...,https://open.spotify.com/album/0k2JGzYRZQC3s3Z...,13.0,ARF102400120,https://open.spotify.com/track/3BSA7eRvD25S4oX...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,None,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,[{'external_urls': {'spotify': 'https://open.s...,1.0,6.0,196800.0,...,FERXXOCALIPSIS,2023-12-01,day,spotify:album:0lgs2Sa82lyX89nBUWyUy6,[{'external_urls': {'spotify': 'https://open.s...,https://open.spotify.com/album/0lgs2Sa82lyX89n...,10.0,USUM72318216,https://open.spotify.com/track/7bywjHOc0wSjGGb...,NaN
1784,https://p.scdn.co/mp3-preview/30e2a899a94a089d...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,[{'external_urls': {'spotify': 'https://open.s...,1.0,11.0,289533.0,...,Dizzy up the Girl,1998,year,spotify:album:4UMjBXcRqIgMZ1XumU2x5T,[{'external_urls': {'spotify': 'https://open.s...,https://open.spotify.com/album/4UMjBXcRqIgMZ1X...,13.0,USWB19800160,https://open.spotify.com/track/6Qyc6fS4DsZjB2m...,NaN
1785,None,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,[{'external_urls': {'spotify': 'https://open.s...,1.0,5.0,200476.0,...,Terlintas,2023-06-22,day,spotify:album:5K8xqV7MCe3UIfedVAlZSe,[{'external_urls': {'spotify': 'https://open.s...,https://open.spotify.com/album/5K8xqV7MCe3UIfe...,5.0,IDA682300214,https://open.spotify.com/track/7zOVh5fGpEwSbZd...,NaN
1786,https://p.scdn.co/mp3-preview/3a490b1336290577...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",True,track,False,True,[{'external_urls': {'spotify': 'https://open.s...,1.0,3.0,130614.0,...,ÉXODO,2024-06-20,day,spotify:album:5kvw1bAOvzJdNi4hdgfaN

In [164]:
df_items_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1788 entries, 0 to 1787
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   preview_url                   1248 non-null   object 
 1   available_markets             1785 non-null   object 
 2   explicit                      1785 non-null   object 
 3   type                          1785 non-null   object 
 4   episode                       1785 non-null   object 
 5   track                         1785 non-null   object 
 6   artists                       1785 non-null   object 
 7   disc_number                   1785 non-null   float64
 8   track_number                  1785 non-null   float64
 9   duration_ms                   1785 non-null   float64
 10  href                          1785 non-null   object 
 11  id                            1785 non-null   object 
 12  name                          1785 non-null   object 
 13  pop

array([<class 'list'>, <class 'float'>], dtype=object)

    available_markets  popularity
638               NaN         NaN
645               NaN         NaN
647               NaN         NaN


,preview_url,available_markets,explicit,type,episode,track,artists,disc_number,track_number,duration_ms,...,album.name,album.release_date,album.release_date_precision,album.uri,album.artists,album.external_urls.spotify,album.total_tracks,external_ids.isrc,external_urls.spotify,external_ids.spotify
4,https://p.scdn.co/mp3-preview/54bba76c1b34f181...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,[{'external_urls': {'spotify': 'https://open.s...,1.0,1.0,104566.0,...,La Lógica del Escorpión,2024-09-12,day,spotify:album:0k2JGzYRZQC3s3ZNTcyHoA,[{'external_urls': {'spotify': 'https://open.s...,https://open.spotify.com/album/0k2JGzYRZQC3s3Z...,13.0,ARF102400120,https://open.spotify.com/track/3BSA7eRvD25S4oX...,NaN
7,https://p.scdn.co/mp3-preview/4b1ded9b01b63fdb...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,[{'external_urls': {'spotify': 'https://open.s...,1.0,2.0,123288.0,...,La Lógica del Escorpión,2024-09-12,day,spotify:album:0k2JGzYRZQC3s3ZNTcyHoA,[{'external_urls': {'spotify': 'https://open.s...,https://open.spotify.com/album/0k2JGzYRZQC3s3Z...,13.0,ARF102400121,https://open.spotify.com/track/79thbxA6OpIUMVu...,NaN
8,https://p.scdn.co/mp3-preview/776ccd62e6b5a8ce...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,[{'external_urls': {'spotify': 'https://open.s...,1.0,3.0,213358.0,...,La Lógica del Escorpión,2024-09-12,day,spotify:album:0k2JGzYRZQC3s3ZNTcyHoA,[{'external_urls': {'spotify': 'https://open.s...,https://open.spotify.com/album/0k2JGzYRZQC3s3Z...,13.0,ARF102400122,https://open.spotify.com/track/7tgHjgeAYcXQCKE...,NaN
11,https://p.scdn.co/mp3-preview/2d70c1a93174807c...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,[{'external_urls': {'spotify': 'https://open.s...,1.0,4.0,225783.0,...,La Lógica del Escorpión,2024-09-12,day,spotify:album:0k2JGzYRZQC3s3ZNTcyHoA,[{'external_urls': {'spotify': 'https://open.s...,https://open.spotify.com/album/0k2JGzYRZQC3s3Z...,13.0,ARF102400123,https://open.spotify.com/track/3yN6KXgZZfe6SVj...,NaN
15,https://p.scdn.co/mp3-preview/9e57f7e37a66c60f...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,[{'external_urls': {'spotify': 'https://open.s...,1.0,5.0,144042.0,...,La Lógica del Escorpión,2024-09-12,day,spotify:album:0k2JGzYRZQC3s3ZNTcyHoA,[{'external_urls': {'spotify': 'https://open.s...,https://open.spotify.com/album/0k2JGzYRZQC3s3Z...,13.0,ARF102400124,https://open.spotify.com/track/1WNvt80Eh9glksp...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,https://p.scdn.co/mp3-preview/64b2faa46e72dad8...,[],False,track,False,True,[{'external_urls': {'spotify': 'https://open.s...,1.0,1.0,442525.0,...,Realm Of Consciousness Pt. III,2018-06-29,day,spotify:album:0obMiSgdjrQh66SkJp6Hlo,[{'external_urls': {'spotify': 'https://open.s...,https://open.spotify.com/album/0obMiSgdjrQh66S...,15.0,DEEC31850079,https://open.spotify.com/track/0uKKDJ3L2ayyhrr...,NaN
1724,https://p.scdn.co/mp3-preview/66a9a43deada4d0f...,[],False,track,False,True,[{'external_urls': {'spotify': 'https://open.s...,1.0,2.0,424561.0,...,Everyday,2012-03-09,day,spotify:album:4SJPfzvRCuJgnJIQzZMl9h,[{'external_urls': {'spotify': 'https://open.s...,https://open.spotify.com/album/4SJPfzvRCuJgnJI...,2.0,DEGL61200004,https://open.spotify.com/track/1vEq5crvEWufZrF...,NaN
1725,https://p.scdn.co/mp3-preview/f42c8cd71015b2f4...,[],False,track,False,True,[{'external_urls': {'spotify': 'https://open.s...,1.0,1.0,390433.0,...,Cream,2012-02-13,day,spotify:album:1qWb0LhxtofniDHzHRnAnO,[{'external_urls': {'spotify': 'https://open.s...,https://open.spotify.com/album/1qWb0LhxtofniDH...,2.0,DEGL61200001,https://open.spotify.com/track/4g8zoFLyGkFRySh...,NaN
1736,https://p.scdn.co/mp3-preview/9245ee570c148895...,[],False,track,False,True,[{'external_urls': {'spotify': 'https://open.s...,1.0,1.0,356254.0,...,Lid (Zimmer Remix),2017-08-25,day,spotify:album:7puHwv1bP

,available_markets,popularity
612,[],0.0
646,[],37.0
648,[],40.0
734,[],1.0
736,[],0.0
...,...,...
1723,[],0.0
1724,[],0.0
1725,[],0.0
1736,[],0.0


87


(1680, 32)

Se eliminaron del dataset las filas con nulos en la variable popularity. Ademas de las que tienen la columna vailable_markets vacia ya que se pueden considerar erroneos

In [165]:
# Limpio las columnas que se por documentacion de la API que no son necesarias para el analisis. Ya que no agregan información acerca de la cancion

columns_to_drop = ['preview_url',
 'track',
 'type',
 'episode',
 'href',
 'uri',
 'is_local',
 'album.available_markets', 
 'album.type',
 'album.href',
 'album.images',
 'album.uri',
 'album.external_urls.spotify',
 'external_ids.isrc',
 'external_urls.spotify',
 'album.artists',
 'external_ids.spotify',
 'album.release_date_precision']

# Dropear las columnas del DataFrame
tracks_top50 = df_items_expanded.drop(columns=columns_to_drop)
tracks_top50


,available_markets,explicit,artists,disc_number,track_number,duration_ms,id,name,popularity,album.album_type,album.id,album.name,album.release_date,album.total_tracks
0,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,[{'external_urls': {'spotify': 'https://open.s...,1.0,1.0,195824.0,6WatFBLVB0x077xWeoVc2k,Si Antes Te Hubiera Conocido,96.0,single,5ylbxH7EqpsmHZCRuiYewS,Si Antes Te Hubiera Conocido,2024-06-21,1.0
1,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,[{'external_urls': {'spotify': 'https://open.s...,1.0,1.0,198495.0,0SoBfYjQHvL6cuWHtPUYnq,Hoy,82.0,single,4ENi7P1IV01E2jzbHZeKxn,Hoy,2024-05-22,1.0
2,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,[{'external_urls': {'spotify': 'https://open.s...,1.0,1.0,144148.0,5DSMm0BZXlSvqArsTX00Ge,DOCTOR,74.0,single,3Z9VnhuHM0BmsNUVPWMd7y,DOCTOR,2024-08-29,1.0
3,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,[{'external_urls': {'spotify': 'https://open.s...,1.0,5.0,165054.0,0c82YIGpQ5dCaXpTNCk4sb,Alegría,72.0,album,3s6jmSUxBLRxzKdevnFmJS,GOTTI A,2024-07-04,14.0
4,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,[{'external_urls': {'spotify': 'https://open.s...,1.0,1.0,104566.0,3BSA7eRvD25S4oX4rdngpU,Rompela,0.0,album,0k2JGzYRZQC3s3ZNTcyHoA,La Lógica del Escorpión,2024-09-12,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,[{'external_urls': {'spotify': 'https://open.s...,1.0,6.0,196800.0,7bywjHOc0wSjGGbj04XbVi,LUNA,90.0,album,0lgs2Sa82lyX89nBUWyUy6,FERXXOCALIPSIS,2023-12-01,10.0
1784,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,[{'external_urls': {'spotify': 'https://open.s...,1.0,11.0,289533.0,6Qyc6fS4DsZjB2mRW9DsQs,Iris,88.0,album,4UMjBXcRqIgMZ1XumU2x5T,Dizzy up the Girl,1998,13.0
1785,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,[{'external_urls': {'spotify': 'https://open.s...,1.0,5.0,200476.0,7zOVh5fGpEwSbZd0g9z80B,Satu Bulan,87.0,single,5K8xqV7MCe3UIfedVAlZSe,Terlintas,2023-06-22,5.0
1786,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",True,[{'external_urls': {'spotify': 'https://open.s...,1.0,3.0,130614.0,2aYZaN5SmkRDLsrrV8GkBQ,LA PATRULLA,87.0,album,5kvw1bAOvzJdNi4hdgfaNf,ÉXODO,2024-06-20,24.0


In [166]:
#Para el JSON de la columna 'artist' me interesa el artista principal y la cantidad de artistas de la cancion
artistas = pd.json_normalize(tracks_top50['artists'])
cant_artistas = artistas.notna().sum(axis=1)

tracks_top50['Cantidad artistas'] = cant_artistas
tracks_top50['artista'] = artistas[0]
tracks_top50 = tracks_top50.drop(columns='artists')
tracks_top50


,available_markets,explicit,disc_number,track_number,duration_ms,id,name,popularity,album.album_type,album.id,album.name,album.release_date,album.total_tracks,Cantidad artistas,artista
0,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,1.0,195824.0,6WatFBLVB0x077xWeoVc2k,Si Antes Te Hubiera Conocido,96.0,single,5ylbxH7EqpsmHZCRuiYewS,Si Antes Te Hubiera Conocido,2024-06-21,1.0,1,{'href': 'https://api.spotify.com/v1/artists/7...
1,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,1.0,198495.0,0SoBfYjQHvL6cuWHtPUYnq,Hoy,82.0,single,4ENi7P1IV01E2jzbHZeKxn,Hoy,2024-05-22,1.0,2,{'href': 'https://api.spotify.com/v1/artists/5...
2,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,1.0,144148.0,5DSMm0BZXlSvqArsTX00Ge,DOCTOR,74.0,single,3Z9VnhuHM0BmsNUVPWMd7y,DOCTOR,2024-08-29,1.0,2,{'href': 'https://api.spotify.com/v1/artists/4...
3,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,5.0,165054.0,0c82YIGpQ5dCaXpTNCk4sb,Alegría,72.0,album,3s6jmSUxBLRxzKdevnFmJS,GOTTI A,2024-07-04,14.0,3,{'href': 'https://api.spotify.com/v1/artists/5...
4,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,1.0,104566.0,3BSA7eRvD25S4oX4rdngpU,Rompela,0.0,album,0k2JGzYRZQC3s3ZNTcyHoA,La Lógica del Escorpión,2024-09-12,13.0,1,{'href': 'https://api.spotify.com/v1/artists/3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,6.0,196800.0,7bywjHOc0wSjGGbj04XbVi,LUNA,90.0,album,0lgs2Sa82lyX89nBUWyUy6,FERXXOCALIPSIS,2023-12-01,10.0,2,{'href': 'https://api.spotify.com/v1/artists/2...
1784,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,11.0,289533.0,6Qyc6fS4DsZjB2mRW9DsQs,Iris,88.0,album,4UMjBXcRqIgMZ1XumU2x5T,Dizzy up the Girl,1998,13.0,1,{'href': 'https://api.spotify.com/v1/artists/2...
1785,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,5.0,200476.0,7zOVh5fGpEwSbZd0g9z80B,Satu Bulan,87.0,single,5K8xqV7MCe3UIfedVAlZSe,Terlintas,2023-06-22,5.0,1,{'href': 'https://api.spotify.com/v1/artists/4...
1786,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",True,1.0,3.0,130614.0,2aYZaN5SmkRDLsrrV8GkBQ,LA PATRULLA,87.0,album,5kvw1bAOvzJdNi4hdgfaNf,ÉXODO,2024-06-20,24.0,2,{'href': 'https://api.spotify.com/v1/artists/1...


In [167]:
artista_expanded = pd.json_normalize(tracks_top50['artista'])
tracks_top50['artist_name'] = artista_expanded['name']
tracks_top50 = tracks_top50.drop(columns='artista')
tracks_top50

,available_markets,explicit,disc_number,track_number,duration_ms,id,name,popularity,album.album_type,album.id,album.name,album.release_date,album.total_tracks,Cantidad artistas,artist_name
0,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,1.0,195824.0,6WatFBLVB0x077xWeoVc2k,Si Antes Te Hubiera Conocido,96.0,single,5ylbxH7EqpsmHZCRuiYewS,Si Antes Te Hubiera Conocido,2024-06-21,1.0,1,KAROL G
1,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,1.0,198495.0,0SoBfYjQHvL6cuWHtPUYnq,Hoy,82.0,single,4ENi7P1IV01E2jzbHZeKxn,Hoy,2024-05-22,1.0,2,Valentino Merlo
2,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,1.0,144148.0,5DSMm0BZXlSvqArsTX00Ge,DOCTOR,74.0,single,3Z9VnhuHM0BmsNUVPWMd7y,DOCTOR,2024-08-29,1.0,2,Luck Ra
3,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,5.0,165054.0,0c82YIGpQ5dCaXpTNCk4sb,Alegría,72.0,album,3s6jmSUxBLRxzKdevnFmJS,GOTTI A,2024-07-04,14.0,3,Tiago PZK
4,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,1.0,104566.0,3BSA7eRvD25S4oX4rdngpU,Rompela,0.0,album,0k2JGzYRZQC3s3ZNTcyHoA,La Lógica del Escorpión,2024-09-12,13.0,1,Charly García
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,6.0,196800.0,7bywjHOc0wSjGGbj04XbVi,LUNA,90.0,album,0lgs2Sa82lyX89nBUWyUy6,FERXXOCALIPSIS,2023-12-01,10.0,2,Feid
1784,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,11.0,289533.0,6Qyc6fS4DsZjB2mRW9DsQs,Iris,88.0,album,4UMjBXcRqIgMZ1XumU2x5T,Dizzy up the Girl,1998,13.0,1,The Goo Goo Dolls
1785,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,5.0,200476.0,7zOVh5fGpEwSbZd0g9z80B,Satu Bulan,87.0,single,5K8xqV7MCe3UIfedVAlZSe,Terlintas,2023-06-22,5.0,1,Bernadya
1786,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",True,1.0,3.0,130614.0,2aYZaN5SmkRDLsrrV8GkBQ,LA PATRULLA,87.0,album,5kvw1bAOvzJdNi4hdgfaNf,ÉXODO,2024-06-20,24.0,2,Peso Pluma


In [169]:
tipos_available_markets = tracks_top50['available_markets'].apply(type).unique()

# Mostrar los tipos de datos
tipos_available_markets

array([<class 'list'>, <class 'float'>], dtype=object)

In [170]:
df_float_markets = tracks_top50[tracks_top50['popularity'].isnull()]

# Seleccionar las columnas 'available_markets' y 'popularity'
df_float_markets = df_float_markets[['available_markets', 'popularity']]

# Mostrar el DataFrame resultante
print(df_float_markets)

    available_markets  popularity
638               NaN         NaN
645               NaN         NaN
647               NaN         NaN


In [171]:
tracks_top50 = tracks_top50.dropna(subset=['popularity'])

In [173]:
popularity_0 = tracks_top50[tracks_top50['popularity'] == 0]
popularity_0

,available_markets,explicit,disc_number,track_number,duration_ms,id,name,popularity,album.album_type,album.id,album.name,album.release_date,album.total_tracks,Cantidad artistas,artist_name
4,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,1.0,104566.0,3BSA7eRvD25S4oX4rdngpU,Rompela,0.0,album,0k2JGzYRZQC3s3ZNTcyHoA,La Lógica del Escorpión,2024-09-12,13.0,1,Charly García
7,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,2.0,123288.0,79thbxA6OpIUMVu8mROice,Yo Ya Sé,0.0,album,0k2JGzYRZQC3s3ZNTcyHoA,La Lógica del Escorpión,2024-09-12,13.0,1,Charly García
8,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,3.0,213358.0,7tgHjgeAYcXQCKEyGpcRj7,El Club de los 27,0.0,album,0k2JGzYRZQC3s3ZNTcyHoA,La Lógica del Escorpión,2024-09-12,13.0,2,Charly García
11,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,4.0,225783.0,3yN6KXgZZfe6SVjRytj5Z2,La Medicina Nº 9,0.0,album,0k2JGzYRZQC3s3ZNTcyHoA,La Lógica del Escorpión,2024-09-12,13.0,2,Charly García
15,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,1.0,5.0,144042.0,1WNvt80Eh9glkspXEEhl6W,Te Recuerdo Invierno,0.0,album,0k2JGzYRZQC3s3ZNTcyHoA,La Lógica del Escorpión,2024-09-12,13.0,1,Charly García
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,[],False,1.0,1.0,442525.0,0uKKDJ3L2ayyhrrhT4iI9v,Nova,0.0,compilation,0obMiSgdjrQh66SkJp6Hlo,Realm Of Consciousness Pt. III,2018-06-29,15.0,1,Tale Of Us
1724,[],False,1.0,2.0,424561.0,1vEq5crvEWufZrFcx3QiQY,Strange,0.0,single,4SJPfzvRCuJgnJIQzZMl9h,Everyday,2012-03-09,2.0,1,Adana Twins
1725,[],False,1.0,1.0,390433.0,4g8zoFLyGkFRyShKXFzzhy,Cream,0.0,single,1qWb0LhxtofniDHzHRnAnO,Cream,2012-02-13,2.0,1,Claptone
1736,[],False,1.0,1.0,356254.0,2vlzVX3TQnaCg69C8MFFee,Lid (Zimmer Remix),0.0,single,7puHwv1bPK5TWLTKBWLBHF,Lid (Zimmer Remix),2017-08-25,1.0,2,Christian Löffler


In [174]:
df_empty_markets = tracks_top50[tracks_top50['available_markets'].apply(lambda x: x == [])]
df_empty_markets[['available_markets', 'popularity']]

,available_markets,popularity
612,[],0.0
646,[],37.0
648,[],40.0
734,[],1.0
736,[],0.0
...,...,...
1723,[],0.0
1724,[],0.0
1725,[],0.0
1736,[],0.0


In [175]:
count_popularity_zero = df_empty_markets[df_empty_markets['popularity'] == 0].shape[0]

print(count_popularity_zero)

87


In [176]:
df_filtered = tracks_top50[~((tracks_top50['available_markets'].apply(lambda x: x == [])))]

In [178]:
df_filtered.shape

(1680, 15)

In [179]:
tracks_top50 = df_filtered.drop(columns=['available_markets'])
tracks_top50

,explicit,disc_number,track_number,duration_ms,id,name,popularity,album.album_type,album.id,album.name,album.release_date,album.total_tracks,Cantidad artistas,artist_name
0,False,1.0,1.0,195824.0,6WatFBLVB0x077xWeoVc2k,Si Antes Te Hubiera Conocido,96.0,single,5ylbxH7EqpsmHZCRuiYewS,Si Antes Te Hubiera Conocido,2024-06-21,1.0,1,KAROL G
1,False,1.0,1.0,198495.0,0SoBfYjQHvL6cuWHtPUYnq,Hoy,82.0,single,4ENi7P1IV01E2jzbHZeKxn,Hoy,2024-05-22,1.0,2,Valentino Merlo
2,False,1.0,1.0,144148.0,5DSMm0BZXlSvqArsTX00Ge,DOCTOR,74.0,single,3Z9VnhuHM0BmsNUVPWMd7y,DOCTOR,2024-08-29,1.0,2,Luck Ra
3,False,1.0,5.0,165054.0,0c82YIGpQ5dCaXpTNCk4sb,Alegría,72.0,album,3s6jmSUxBLRxzKdevnFmJS,GOTTI A,2024-07-04,14.0,3,Tiago PZK
4,False,1.0,1.0,104566.0,3BSA7eRvD25S4oX4rdngpU,Rompela,0.0,album,0k2JGzYRZQC3s3ZNTcyHoA,La Lógica del Escorpión,2024-09-12,13.0,1,Charly García
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,False,1.0,6.0,196800.0,7bywjHOc0wSjGGbj04XbVi,LUNA,90.0,album,0lgs2Sa82lyX89nBUWyUy6,FERXXOCALIPSIS,2023-12-01,10.0,2,Feid
1784,False,1.0,11.0,289533.0,6Qyc6fS4DsZjB2mRW9DsQs,Iris,88.0,album,4UMjBXcRqIgMZ1XumU2x5T,Dizzy up the Girl,1998,13.0,1,The Goo Goo Dolls
1785,False,1.0,5.0,200476.0,7zOVh5fGpEwSbZd0g9z80B,Satu Bulan,87.0,single,5K8xqV7MCe3UIfedVAlZSe,Terlintas,2023-06-22,5.0,1,Bernadya
1786,True,1.0,3.0,130614.0,2aYZaN5SmkRDLsrrV8GkBQ,LA PATRULLA,87.0,album,5kvw1bAOvzJdNi4hdgfaNf,ÉXODO,2024-06-20,24.0,2,Peso Pluma


In [180]:
#Agrego una columna con los dias desde que salio el album (la cancion)

tracks_top50['album.release_date'] = pd.to_datetime(tracks_top50['album.release_date'])
today = pd.to_datetime(datetime.now().date())
tracks_top50['days_since_release'] = (today - tracks_top50['album.release_date']).dt.days
tracks_top50

,explicit,disc_number,track_number,duration_ms,id,name,popularity,album.album_type,album.id,album.name,album.release_date,album.total_tracks,Cantidad artistas,artist_name,days_since_release
0,False,1.0,1.0,195824.0,6WatFBLVB0x077xWeoVc2k,Si Antes Te Hubiera Conocido,96.0,single,5ylbxH7EqpsmHZCRuiYewS,Si Antes Te Hubiera Conocido,2024-06-21,1.0,1,KAROL G,84.0
1,False,1.0,1.0,198495.0,0SoBfYjQHvL6cuWHtPUYnq,Hoy,82.0,single,4ENi7P1IV01E2jzbHZeKxn,Hoy,2024-05-22,1.0,2,Valentino Merlo,114.0
2,False,1.0,1.0,144148.0,5DSMm0BZXlSvqArsTX00Ge,DOCTOR,74.0,single,3Z9VnhuHM0BmsNUVPWMd7y,DOCTOR,2024-08-29,1.0,2,Luck Ra,15.0
3,False,1.0,5.0,165054.0,0c82YIGpQ5dCaXpTNCk4sb,Alegría,72.0,album,3s6jmSUxBLRxzKdevnFmJS,GOTTI A,2024-07-04,14.0,3,Tiago PZK,71.0
4,False,1.0,1.0,104566.0,3BSA7eRvD25S4oX4rdngpU,Rompela,0.0,album,0k2JGzYRZQC3s3ZNTcyHoA,La Lógica del Escorpión,2024-09-12,13.0,1,Charly García,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,False,1.0,6.0,196800.0,7bywjHOc0wSjGGbj04XbVi,LUNA,90.0,album,0lgs2Sa82lyX89nBUWyUy6,FERXXOCALIPSIS,2023-12-01,10.0,2,Feid,287.0
1784,False,1.0,11.0,289533.0,6Qyc6fS4DsZjB2mRW9DsQs,Iris,88.0,album,4UMjBXcRqIgMZ1XumU2x5T,Dizzy up the Girl,1998-01-01,13.0,1,The Goo Goo Dolls,9752.0
1785,False,1.0,5.0,200476.0,7zOVh5fGpEwSbZd0g9z80B,Satu Bulan,87.0,single,5K8xqV7MCe3UIfedVAlZSe,Terlintas,2023-06-22,5.0,1,Bernadya,449.0
1786,True,1.0,3.0,130614.0,2aYZaN5SmkRDLsrrV8GkBQ,LA PATRULLA,87.0,album,5kvw1bAOvzJdNi4hdgfaNf,ÉXODO,2024-06-20,24.0,2,Peso Pluma,85.0


In [181]:
# Contar la cantidad de veces que aparece cada ID
id_counts = tracks_top50['id'].value_counts()

# Filtrar para mostrar solo los IDs que aparecen más de una vez
duplicate_ids = id_counts[id_counts > 1]
duplicate_ids

2plbrEY59IikOBgBGLjaoe    9
6dOtVTDdiauQNBQEDOtlAB    8
2PnlsTsOTLE5jnBnNe2K0A    7
6WatFBLVB0x077xWeoVc2k    6
2qSkIjg1o9h3YT9RAgYN75    6
                         ..
0AjmK0Eai4zGrLaJwPvrDp    2
2Dct3GykKZ58hpWRFfe2Qd    2
5jzma6gCzYtKB1DbEwFZKH    2
4YnmQG1fnJR9LnTZtix70T    2
5iJKGpnFfvbjZJeAtwXfCj    2
Name: id, Length: 120, dtype: int64

In [182]:
#Es posible que dos canciones esten en dos playlist distintas de cada pais, por lo que elimino duplicados por id

final_tracks = tracks_top50.drop_duplicates(subset='id')
final_tracks = final_tracks.reset_index(drop=True)
final_tracks

,explicit,disc_number,track_number,duration_ms,id,name,popularity,album.album_type,album.id,album.name,album.release_date,album.total_tracks,Cantidad artistas,artist_name,days_since_release
0,False,1.0,1.0,195824.0,6WatFBLVB0x077xWeoVc2k,Si Antes Te Hubiera Conocido,96.0,single,5ylbxH7EqpsmHZCRuiYewS,Si Antes Te Hubiera Conocido,2024-06-21,1.0,1,KAROL G,84.0
1,False,1.0,1.0,198495.0,0SoBfYjQHvL6cuWHtPUYnq,Hoy,82.0,single,4ENi7P1IV01E2jzbHZeKxn,Hoy,2024-05-22,1.0,2,Valentino Merlo,114.0
2,False,1.0,1.0,144148.0,5DSMm0BZXlSvqArsTX00Ge,DOCTOR,74.0,single,3Z9VnhuHM0BmsNUVPWMd7y,DOCTOR,2024-08-29,1.0,2,Luck Ra,15.0
3,False,1.0,5.0,165054.0,0c82YIGpQ5dCaXpTNCk4sb,Alegría,72.0,album,3s6jmSUxBLRxzKdevnFmJS,GOTTI A,2024-07-04,14.0,3,Tiago PZK,71.0
4,False,1.0,1.0,104566.0,3BSA7eRvD25S4oX4rdngpU,Rompela,0.0,album,0k2JGzYRZQC3s3ZNTcyHoA,La Lógica del Escorpión,2024-09-12,13.0,1,Charly García,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,False,1.0,12.0,183956.0,5XeFesFbtLpXzIVDNQP22n,I Wanna Be Yours,91.0,album,78bpIziExqiI9qztvNFlQu,AM,2013-09-09,12.0,1,Arctic Monkeys,4022.0
1456,False,1.0,1.0,200400.0,62bOmKYxYg7dhrC6gH9vFn,Bye Bye Bye - From Deadpool and Wolverine Soun...,90.0,album,20RMokVwJ2wjQ0s8FOdOFC,No Strings Attached,2000-03-21,12.0,1,*NSYNC,8942.0
1457,True,1.0,8.0,191700.0,42VsgItocQwOQC3XWZ8JNA,FE!N (feat. Playboi Carti),90.0,album,18NOKLkZETa4sWwLMIm0UZ,UTOPIA,2023-07-28,19.0,2,Travis Scott,413.0
1458,False,1.0,6.0,159245.0,3qhlB30KknSejmIvZZLjOD,End of Beginning,92.0,album,4JDXBZdRPId4devseaAOKH,DECIDE,2022-09-16,13.0,1,Djo,728.0


In [183]:
# Verificamos si hay ids de cancion nulos
final_tracks['id'].isnull().sum()

0

In [184]:
# Hay un valor nulo, por lo que lo vamos a buscar y ver que paso
final_tracks[final_tracks['id'].isnull()]

,explicit,disc_number,track_number,duration_ms,id,name,popularity,album.album_type,album.id,album.name,album.release_date,album.total_tracks,Cantidad artistas,artist_name,days_since_release


In [185]:
# Dropeamos la columna con ID nulo

final_tracks = final_tracks.dropna(subset=['id'])

In [186]:
final_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   explicit            1460 non-null   object        
 1   disc_number         1460 non-null   float64       
 2   track_number        1460 non-null   float64       
 3   duration_ms         1460 non-null   float64       
 4   id                  1460 non-null   object        
 5   name                1460 non-null   object        
 6   popularity          1460 non-null   float64       
 7   album.album_type    1460 non-null   object        
 8   album.id            1460 non-null   object        
 9   album.name          1460 non-null   object        
 10  album.release_date  1454 non-null   datetime64[ns]
 11  album.total_tracks  1460 non-null   float64       
 12  Cantidad artistas   1460 non-null   int64         
 13  artist_name         1454 non-null   object      

In [187]:
# Obtener los datos de la columna 'id' y convertirlos en una lista
track_ids = final_tracks['id'].tolist()

#Divido la lista en partes iguales ya que la API no me permite pedir mas de 100 datos a la vez
cantidad_filas = len(final_tracks)

resultado = np.ceil(cantidad_filas / 100)

split_data = np.array_split(track_ids, resultado)
split_data

[array(['6WatFBLVB0x077xWeoVc2k', '0SoBfYjQHvL6cuWHtPUYnq',
        '5DSMm0BZXlSvqArsTX00Ge', '0c82YIGpQ5dCaXpTNCk4sb',
        '3BSA7eRvD25S4oX4rdngpU', '4dLLdfiniBhDP9dlJQIOi4',
        '6kPCcIAGoTeRrh8qrmKa90', '79thbxA6OpIUMVu8mROice',
        '7tgHjgeAYcXQCKEyGpcRj7', '5rQSQlZXXjMcevPGoAfE1z',
        '3KeYEr9ogMBemsOfQIJmZp', '3yN6KXgZZfe6SVjRytj5Z2',
        '2AFC9ad4nGauS1cgm7FCqU', '42wdz3j7gstgUuhoFLCXqd',
        '1HRUB9L0IRcRHzLVD4pk6C', '1WNvt80Eh9glkspXEEhl6W',
        '2N0hVk2Ant8751xPZUjCV2', '23syx1ODrqa0BkQzaaHd3r',
        '3cWo162YkfLIFyB3DWbVoH', '51Mo6woPjQEPOON5eWc220',
        '1TyBUtMxdmRfXhW3MmbxtQ', '5AkLEnorFvb7WIJOGqKYVT',
        '1pymWRCuZfCd0zdiBJo0Hj', '6srEY1lFkB0S3mkNbL5BDT',
        '5QjmUqgpPQgXgg4606DqZF', '2TBoLwjf3gSWuVa2Bh8Ayv',
        '4YnmQG1fnJR9LnTZtix70T', '2RmtEc2ZNDCzAU3vH0AbKx',
        '7bywjHOc0wSjGGbj04XbVi', '42mrOMjkQb7jRKatu0bqg4',
        '54e9B1fVeKg8qBLi4ysSXA', '4hMpKCpWbYroQ8VKlABGk4',
        '7Je1Wx6gRPt4dhnbgaEAOu', '0Cs2A

In [188]:
print(f"IDs en el lote 6: {split_data[5]}")


IDs en el lote 6: ['5FE96VMbxbodJP59XMxHhl' '2wOuPlBMtQbvLEmgDlnaoo'
 '2N68XdwQ97tF145Ska3g44' '6aYjxfCm15W0ez6rCa0XZL'
 '4XGmH1PRv5I56MllQinZtD' '4Ueyqrz4v0UzqDupbxByqx'
 '1OpJNP2QNZ78TnewxSVybt' '1edgaGx2x2AQeWtL2OPEMW'
 '3uwYgNxFDfx1GoLB6tLoUn' '2nG54Y4a3sH9YpfxMolOyi'
 '0wEGo9LsSrSUz9HOtraWAO' '7kcApwUclbIw9B6MuVmqPc'
 '2tkKKCiYliwLh7sHARM4t8' '4UNqOGV5RNfVjB62iTa0k0'
 '1RIzTUUBezI4zjONsEyR9M' '5XnbN4pz5jdnxyh4iZiQ93'
 '10Jq6BTxOzwTylho4Dan5C' '6uXvGMgYFnTG8ztC2yI2Id'
 '7GVQS66ukm48XZVymA3ZUg' '19CCQeIOxxUDcpFw539zd7'
 '5S1vJejm1YukfhXa5PxPan' '2dDjXlJa6pnOTGl8C8iChL'
 '7uBmzYZIqTogjFZq3cC6kf' '2icnkjMudyHcjCHbxvYo2l'
 '2s420tfpQ4lY5wy9JsRCxi' '7oJ3Nb3LIY1ond1fHF3xio'
 '5woNfSOnZYbFbA94DqvJJG' '588EtZ99qVxlMWiqRnc41x'
 '5hKE4hbAVMP38XWjRlXZPO' '1RYQogjK7bMYacOrQrq0CL'
 '2kDApipZtTzjwGfKujcg2z' '2LlfzwOBZxHiEkdFc62Hgy'
 '4zEeVJMVdmPjP5Hw9NR9Ou' '53IRnAWx13PYmoVYtemUBS'
 '0Cg1vTBSkpoSYz6LxYxrre' '6ebIXbJD7VVZNSgeuGQFfO'
 '0vSGGbwIZE8tjiiOQorrGW' '7b4iOHMngSAFhgN120hbWB'
 '1Jq6a9u1fZ6

In [189]:
# Función para obtener características de audio
def get_audio_features(track_ids):
    # Convertir la lista de IDs en una cadena separada por comas
    ids_string = '%2C'.join(track_ids)
    
    # URL del endpoint de Spotify para obtener las características de audio
    url = f"https://api.spotify.com/v1/audio-features?ids={ids_string}"
    
    # Hacer la solicitud GET
    response = requests.get(url, headers=headers)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        try:
            response_json = response.json()
        except ValueError:
            print("Error: No se pudo decodificar el JSON de la respuesta.")
            return None
        
        # Verificar si 'audio_features' está presente y no es None
        if response_json.get('audio_features'):
            return response_json['audio_features']
        else:
            print(f"No se encontraron 'audio_features' en la respuesta para los IDs: {track_ids}")
            print("Respuesta recibida:", response_json)
            return None
    else:
        print(f"Error en la solicitud: {response.status_code}")
        print("Respuesta del servidor:", response.text)
        return None

# DataFrame vacío para almacenar todos los resultados
df_all_features = pd.DataFrame()

# Iterar sobre cada lista de IDs
for i in range(0, len(split_data)):
    print(f"Solicitando características de audio para lote {i+1} de {len(split_data)}...")
    
    # Obtener las características de audio para la lista actual de IDs
    features_data = get_audio_features(split_data[i])
    
    # Validar si 'features_data' no es None antes de procesar
    if features_data:
        # Convertir la respuesta en un DataFrame
        try:
            df_each_feature = pd.DataFrame(features_data)
            # Concatenar el DataFrame actual con el DataFrame global
            df_all_features = pd.concat([df_all_features, df_each_feature], ignore_index=True)
        except Exception as e:
            print(f"Error al crear el DataFrame: {e}")
    else:
        print(f"No se encontraron características para el lote {i+1}")

df_all_features


Solicitando características de audio para lote 1 de 15...
Solicitando características de audio para lote 2 de 15...
Solicitando características de audio para lote 3 de 15...
Solicitando características de audio para lote 4 de 15...
Solicitando características de audio para lote 5 de 15...
Solicitando características de audio para lote 6 de 15...
Error al crear el DataFrame: 'NoneType' object has no attribute 'keys'
Solicitando características de audio para lote 7 de 15...
Solicitando características de audio para lote 8 de 15...
Solicitando características de audio para lote 9 de 15...
Solicitando características de audio para lote 10 de 15...
Solicitando características de audio para lote 11 de 15...
Solicitando características de audio para lote 12 de 15...
Solicitando características de audio para lote 13 de 15...
Solicitando características de audio para lote 14 de 15...
Solicitando características de audio para lote 15 de 15...


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.924,0.668,11,-6.795,1,0.0469,0.4460,0.000594,0.0678,0.787,128.027,audio_features,6WatFBLVB0x077xWeoVc2k,spotify:track:6WatFBLVB0x077xWeoVc2k,https://api.spotify.com/v1/tracks/6WatFBLVB0x0...,https://api.spotify.com/v1/audio-analysis/6Wat...,195824,4
1,0.451,0.711,11,-4.612,1,0.0426,0.2400,0.000000,0.1540,0.661,176.161,audio_features,0SoBfYjQHvL6cuWHtPUYnq,spotify:track:0SoBfYjQHvL6cuWHtPUYnq,https://api.spotify.com/v1/tracks/0SoBfYjQHvL6...,https://api.spotify.com/v1/audio-analysis/0SoB...,198496,4
2,0.651,0.762,9,-2.969,1,0.0315,0.0834,0.000000,0.0701,0.721,90.989,audio_features,5DSMm0BZXlSvqArsTX00Ge,spotify:track:5DSMm0BZXlSvqArsTX00Ge,https://api.spotify.com/v1/tracks/5DSMm0BZXlSv...,https://api.spotify.com/v1/audio-analysis/5DSM...,144148,4
3,0.575,0.658,9,-2.340,0,0.1010,0.0463,0.000022,0.0627,0.884,84.273,audio_features,0c82YIGpQ5dCaXpTNCk4sb,spotify:track:0c82YIGpQ5dCaXpTNCk4sb,https://api.spotify.com/v1/tracks/0c82YIGpQ5dC...,https://api.spotify.com/v1/audio-analysis/0c82...,167055,5
4,0.716,0.888,9,-5.167,1,0.0316,0.1540,0.000000,0.3460,0.953,111.465,audio_features,3BSA7eRvD25S4oX4rdngpU,spotify:track:3BSA7eRvD25S4oX4rdngpU,https://api.spotify.com/v1/tracks/3BSA7eRvD25S...,https://api.spotify.com/v1/audio-analysis/3BSA...,104567,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358,0.464,0.417,0,-9.345,0,0.0256,0.1360,0.022000,0.0974,0.479,67.528,audio_features,5XeFesFbtLpXzIVDNQP22n,spotify:track:5XeFesFbtLpXzIVDNQP22n,https://api.spotify.com/v1/tracks/5XeFesFbtLpX...,https://api.spotify.com/v1/audio-analysis/5XeF...,183956,4
1359,0.610,0.926,8,-4.843,0,0.0479,0.0310,0.001200,0.0821,0.861,172.638,audio_features,62bOmKYxYg7dhrC6gH9vFn,spotify:track:62bOmKYxYg7dhrC6gH9vFn,https://api.spotify.com/v1/tracks/62bOmKYxYg7d...,https://api.spotify.com/v1/audio-analysis/62bO...,200400,4
1360,0.569,0.882,3,-2.777,0,0.0600,0.0316,0.000000,0.1420,0.201,148.038,audio_features,42VsgItocQwOQC3XWZ8JNA,spotify:track:42VsgItocQwOQC3XWZ8JNA,https://api.spotify.com/v1/tracks/42VsgItocQwO...,https://api.spotify.com/v1/audio-analysis/42Vs...,191701,4
1361,0.689,0.454,2,-7.643,1,0.0584,0.0351,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246,4


In [190]:
print(f"IDs en el lote 6: {split_data[5]}")

IDs en el lote 6: ['5FE96VMbxbodJP59XMxHhl' '2wOuPlBMtQbvLEmgDlnaoo'
 '2N68XdwQ97tF145Ska3g44' '6aYjxfCm15W0ez6rCa0XZL'
 '4XGmH1PRv5I56MllQinZtD' '4Ueyqrz4v0UzqDupbxByqx'
 '1OpJNP2QNZ78TnewxSVybt' '1edgaGx2x2AQeWtL2OPEMW'
 '3uwYgNxFDfx1GoLB6tLoUn' '2nG54Y4a3sH9YpfxMolOyi'
 '0wEGo9LsSrSUz9HOtraWAO' '7kcApwUclbIw9B6MuVmqPc'
 '2tkKKCiYliwLh7sHARM4t8' '4UNqOGV5RNfVjB62iTa0k0'
 '1RIzTUUBezI4zjONsEyR9M' '5XnbN4pz5jdnxyh4iZiQ93'
 '10Jq6BTxOzwTylho4Dan5C' '6uXvGMgYFnTG8ztC2yI2Id'
 '7GVQS66ukm48XZVymA3ZUg' '19CCQeIOxxUDcpFw539zd7'
 '5S1vJejm1YukfhXa5PxPan' '2dDjXlJa6pnOTGl8C8iChL'
 '7uBmzYZIqTogjFZq3cC6kf' '2icnkjMudyHcjCHbxvYo2l'
 '2s420tfpQ4lY5wy9JsRCxi' '7oJ3Nb3LIY1ond1fHF3xio'
 '5woNfSOnZYbFbA94DqvJJG' '588EtZ99qVxlMWiqRnc41x'
 '5hKE4hbAVMP38XWjRlXZPO' '1RYQogjK7bMYacOrQrq0CL'
 '2kDApipZtTzjwGfKujcg2z' '2LlfzwOBZxHiEkdFc62Hgy'
 '4zEeVJMVdmPjP5Hw9NR9Ou' '53IRnAWx13PYmoVYtemUBS'
 '0Cg1vTBSkpoSYz6LxYxrre' '6ebIXbJD7VVZNSgeuGQFfO'
 '0vSGGbwIZE8tjiiOQorrGW' '7b4iOHMngSAFhgN120hbWB'
 '1Jq6a9u1fZ6

In [191]:
# No se ve ningun problema claro en el lote 6, vamos a ver en los datos corresopndientes
final_tracks.iloc[500:600]

,explicit,disc_number,track_number,duration_ms,id,name,popularity,album.album_type,album.id,album.name,album.release_date,album.total_tracks,Cantidad artistas,artist_name,days_since_release
500,False,1.0,1.0,236000.0,0wEGo9LsSrSUz9HOtraWAO,Overcompensate,64.0,single,5ZSqGFLuXUJUlIObSkN0Bz,Overcompensate,2024-02-29,2.0,1,Twenty One Pilots,197.0
501,False,1.0,2.0,190733.0,7kcApwUclbIw9B6MuVmqPc,Overcompensate (edit),52.0,single,5ZSqGFLuXUJUlIObSkN0Bz,Overcompensate,2024-02-29,2.0,1,Twenty One Pilots,197.0
502,False,0.0,0.0,251147.0,2tkKKCiYliwLh7sHARM4t8,Forest,0.0,compilation,56TOa7cx0sJrGO2hyFa1Pg,RAB,NaT,1.0,1,None,NaN
503,False,0.0,0.0,266472.0,4UNqOGV5RNfVjB62iTa0k0,Glowing Eyes,0.0,compilation,56TOa7cx0sJrGO2hyFa1Pg,RAB,NaT,1.0,1,None,NaN
504,False,0.0,0.0,324173.0,1RIzTUUBezI4zjONsEyR9M,Kitchen Sink,0.0,compilation,56TOa7cx0sJrGO2hyFa1Pg,RAB,NaT,1.0,1,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,False,1.0,12.0,232026.0,3oqWr0jDWNXxWufNogGREp,Crimen,72.0,album,7KHNK7l8peO0t95I1v7BmP,Ahí Vamos,2006-04-04,13.0,1,Gustavo Cerati,6737.0
596,False,1.0,2.0,363120.0,4eCrF1FgVL4se3AkHOnLDL,Mujer Amante,70.0,album,3wNTH6hg5KKjRtLJZHxhB9,"Magos, Espadas Y Rosas",2004-01-01,9.0,1,Rata Blanca,7561.0
597,False,1.0,11.0,313000.0,3ydweVPwq9PCEVJ8WwqT6u,Loco (Tu Forma De Ser),69.0,album,2pqscM7JlS6XR6fk7Y8tVZ,El Milagro Argentino,1989-06-27,13.0,1,Los Auténticos Decadentes,12862.0
598,False,1.0,5.0,233746.0,1rh4kDY9T4fHVDum8Foi5k,Adiós,70.0,album,7KHNK7l8peO0t95I1v7BmP,Ahí Vamos,2006-04-04,13.0,1,Gustavo Cerati,6737.0


### Si el atributo album.album_type es compilation, la API no trae track features, por los que vamos a eliminar estos registros. Además su valor de popularidad es 0

In [192]:
final_tracks = final_tracks[final_tracks['album.album_type'] != 'compilation']

In [193]:
# Nuevamente obtenemos los ids y hacemos el request

track_ids = final_tracks['id'].tolist()
cantidad_filas = len(final_tracks)
resultado = np.ceil(cantidad_filas / 100)
split_data = np.array_split(track_ids, resultado)

# Función para obtener características de audio
def get_audio_features(track_ids):
    # Convertir la lista de IDs en una cadena separada por comas
    ids_string = '%2C'.join(track_ids)
    
    # URL del endpoint de Spotify para obtener las características de audio
    url = f"https://api.spotify.com/v1/audio-features?ids={ids_string}"
    
    # Hacer la solicitud GET
    response = requests.get(url, headers=headers)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        try:
            response_json = response.json()
        except ValueError:
            print("Error: No se pudo decodificar el JSON de la respuesta.")
            return None
        
        # Verificar si 'audio_features' está presente y no es None
        if response_json.get('audio_features'):
            return response_json['audio_features']
        else:
            print(f"No se encontraron 'audio_features' en la respuesta para los IDs: {track_ids}")
            print("Respuesta recibida:", response_json)
            return None
    else:
        print(f"Error en la solicitud: {response.status_code}")
        print("Respuesta del servidor:", response.text)
        return None

# DataFrame vacío para almacenar todos los resultados
df_all_features = pd.DataFrame()

# Iterar sobre cada lista de IDs
for i in range(0, len(split_data)):
    print(f"Solicitando características de audio para lote {i+1} de {len(split_data)}...")
    
    # Obtener las características de audio para la lista actual de IDs
    features_data = get_audio_features(split_data[i])
    
    # Validar si 'features_data' no es None antes de procesar
    if features_data:
        # Convertir la respuesta en un DataFrame
        try:
            df_each_feature = pd.DataFrame(features_data)
            # Concatenar el DataFrame actual con el DataFrame global
            df_all_features = pd.concat([df_all_features, df_each_feature], ignore_index=True)
        except Exception as e:
            print(f"Error al crear el DataFrame: {e}")
    else:
        print(f"No se encontraron características para el lote {i+1}")

df_all_features

Solicitando características de audio para lote 1 de 14...
Solicitando características de audio para lote 2 de 14...
Solicitando características de audio para lote 3 de 14...
Solicitando características de audio para lote 4 de 14...
Solicitando características de audio para lote 5 de 14...
Solicitando características de audio para lote 6 de 14...
Solicitando características de audio para lote 7 de 14...
Solicitando características de audio para lote 8 de 14...
Solicitando características de audio para lote 9 de 14...
Solicitando características de audio para lote 10 de 14...
Solicitando características de audio para lote 11 de 14...
Solicitando características de audio para lote 12 de 14...
Solicitando características de audio para lote 13 de 14...
Solicitando características de audio para lote 14 de 14...


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.924,0.668,11,-6.795,1,0.0469,0.4460,0.000594,0.0678,0.787,128.027,audio_features,6WatFBLVB0x077xWeoVc2k,spotify:track:6WatFBLVB0x077xWeoVc2k,https://api.spotify.com/v1/tracks/6WatFBLVB0x0...,https://api.spotify.com/v1/audio-analysis/6Wat...,195824,4
1,0.451,0.711,11,-4.612,1,0.0426,0.2400,0.000000,0.1540,0.661,176.161,audio_features,0SoBfYjQHvL6cuWHtPUYnq,spotify:track:0SoBfYjQHvL6cuWHtPUYnq,https://api.spotify.com/v1/tracks/0SoBfYjQHvL6...,https://api.spotify.com/v1/audio-analysis/0SoB...,198496,4
2,0.651,0.762,9,-2.969,1,0.0315,0.0834,0.000000,0.0701,0.721,90.989,audio_features,5DSMm0BZXlSvqArsTX00Ge,spotify:track:5DSMm0BZXlSvqArsTX00Ge,https://api.spotify.com/v1/tracks/5DSMm0BZXlSv...,https://api.spotify.com/v1/audio-analysis/5DSM...,144148,4
3,0.575,0.658,9,-2.340,0,0.1010,0.0463,0.000022,0.0627,0.884,84.273,audio_features,0c82YIGpQ5dCaXpTNCk4sb,spotify:track:0c82YIGpQ5dCaXpTNCk4sb,https://api.spotify.com/v1/tracks/0c82YIGpQ5dC...,https://api.spotify.com/v1/audio-analysis/0c82...,167055,5
4,0.716,0.888,9,-5.167,1,0.0316,0.1540,0.000000,0.3460,0.953,111.465,audio_features,3BSA7eRvD25S4oX4rdngpU,spotify:track:3BSA7eRvD25S4oX4rdngpU,https://api.spotify.com/v1/tracks/3BSA7eRvD25S...,https://api.spotify.com/v1/audio-analysis/3BSA...,104567,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,0.464,0.417,0,-9.345,0,0.0256,0.1360,0.022000,0.0974,0.479,67.528,audio_features,5XeFesFbtLpXzIVDNQP22n,spotify:track:5XeFesFbtLpXzIVDNQP22n,https://api.spotify.com/v1/tracks/5XeFesFbtLpX...,https://api.spotify.com/v1/audio-analysis/5XeF...,183956,4
1378,0.610,0.926,8,-4.843,0,0.0479,0.0310,0.001200,0.0821,0.861,172.638,audio_features,62bOmKYxYg7dhrC6gH9vFn,spotify:track:62bOmKYxYg7dhrC6gH9vFn,https://api.spotify.com/v1/tracks/62bOmKYxYg7d...,https://api.spotify.com/v1/audio-analysis/62bO...,200400,4
1379,0.569,0.882,3,-2.777,0,0.0600,0.0316,0.000000,0.1420,0.201,148.038,audio_features,42VsgItocQwOQC3XWZ8JNA,spotify:track:42VsgItocQwOQC3XWZ8JNA,https://api.spotify.com/v1/tracks/42VsgItocQwO...,https://api.spotify.com/v1/audio-analysis/42Vs...,191701,4
1380,0.689,0.454,2,-7.643,1,0.0584,0.0351,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246,4


In [194]:
#Eliminamos las columnas que no seran necesarias

columns = ['type',
 'uri',
 'track_href',
 'analysis_url',
 'duration_ms'] #duration ya figura en el otro df

# Dropear las columnas del DataFrame
features = df_all_features.drop(columns=columns)
features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,time_signature
0,0.924,0.668,11,-6.795,1,0.0469,0.4460,0.000594,0.0678,0.787,128.027,6WatFBLVB0x077xWeoVc2k,4
1,0.451,0.711,11,-4.612,1,0.0426,0.2400,0.000000,0.1540,0.661,176.161,0SoBfYjQHvL6cuWHtPUYnq,4
2,0.651,0.762,9,-2.969,1,0.0315,0.0834,0.000000,0.0701,0.721,90.989,5DSMm0BZXlSvqArsTX00Ge,4
3,0.575,0.658,9,-2.340,0,0.1010,0.0463,0.000022,0.0627,0.884,84.273,0c82YIGpQ5dCaXpTNCk4sb,5
4,0.716,0.888,9,-5.167,1,0.0316,0.1540,0.000000,0.3460,0.953,111.465,3BSA7eRvD25S4oX4rdngpU,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,0.464,0.417,0,-9.345,0,0.0256,0.1360,0.022000,0.0974,0.479,67.528,5XeFesFbtLpXzIVDNQP22n,4
1378,0.610,0.926,8,-4.843,0,0.0479,0.0310,0.001200,0.0821,0.861,172.638,62bOmKYxYg7dhrC6gH9vFn,4
1379,0.569,0.882,3,-2.777,0,0.0600,0.0316,0.000000,0.1420,0.201,148.038,42VsgItocQwOQC3XWZ8JNA,4
1380,0.689,0.454,2,-7.643,1,0.0584,0.0351,0.002590,0.0707,0.912,159.982,3qhlB30KknSejmIvZZLjOD,4


In [195]:
datos = pd.merge(final_tracks, features, on='id', how='outer')
datos

,explicit,disc_number,track_number,duration_ms,id,name,popularity,album.album_type,album.id,album.name,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,False,1.0,1.0,195824.0,6WatFBLVB0x077xWeoVc2k,Si Antes Te Hubiera Conocido,96.0,single,5ylbxH7EqpsmHZCRuiYewS,Si Antes Te Hubiera Conocido,...,11,-6.795,1,0.0469,0.4460,0.000594,0.0678,0.787,128.027,4
1,False,1.0,1.0,198495.0,0SoBfYjQHvL6cuWHtPUYnq,Hoy,82.0,single,4ENi7P1IV01E2jzbHZeKxn,Hoy,...,11,-4.612,1,0.0426,0.2400,0.000000,0.1540,0.661,176.161,4
2,False,1.0,1.0,144148.0,5DSMm0BZXlSvqArsTX00Ge,DOCTOR,74.0,single,3Z9VnhuHM0BmsNUVPWMd7y,DOCTOR,...,9,-2.969,1,0.0315,0.0834,0.000000,0.0701,0.721,90.989,4
3,False,1.0,5.0,165054.0,0c82YIGpQ5dCaXpTNCk4sb,Alegría,72.0,album,3s6jmSUxBLRxzKdevnFmJS,GOTTI A,...,9,-2.340,0,0.1010,0.0463,0.000022,0.0627,0.884,84.273,5
4,False,1.0,1.0,104566.0,3BSA7eRvD25S4oX4rdngpU,Rompela,0.0,album,0k2JGzYRZQC3s3ZNTcyHoA,La Lógica del Escorpión,...,9,-5.167,1,0.0316,0.1540,0.000000,0.3460,0.953,111.465,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,False,1.0,12.0,183956.0,5XeFesFbtLpXzIVDNQP22n,I Wanna Be Yours,91.0,album,78bpIziExqiI9qztvNFlQu,AM,...,0,-9.345,0,0.0256,0.1360,0.022000,0.0974,0.479,67.528,4
1378,False,1.0,1.0,200400.0,62bOmKYxYg7dhrC6gH9vFn,Bye Bye Bye - From Deadpool and Wolverine Soun...,90.0,album,20RMokVwJ2wjQ0s8FOdOFC,No Strings Attached,...,8,-4.843,0,0.0479,0.0310,0.001200,0.0821,0.861,172.638,4
1379,True,1.0,8.0,191700.0,42VsgItocQwOQC3XWZ8JNA,FE!N (feat. Playboi Carti),90.0,album,18NOKLkZETa4sWwLMIm0UZ,UTOPIA,...,3,-2.777,0,0.0600,0.0316,0.000000,0.1420,0.201,148.038,4
1380,False,1.0,6.0,159245.0,3qhlB30KknSejmIvZZLjOD,End of Beginning,92.0,album,4JDXBZdRPId4devseaAOKH,DECIDE,...,2,-7.643,1,0.0584,0.0351,0.002590,0.0707,0.912,159.982,4


In [196]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1382 entries, 0 to 1381
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   explicit            1382 non-null   object        
 1   disc_number         1382 non-null   float64       
 2   track_number        1382 non-null   float64       
 3   duration_ms         1382 non-null   float64       
 4   id                  1382 non-null   object        
 5   name                1382 non-null   object        
 6   popularity          1382 non-null   float64       
 7   album.album_type    1382 non-null   object        
 8   album.id            1382 non-null   object        
 9   album.name          1382 non-null   object        
 10  album.release_date  1382 non-null   datetime64[ns]
 11  album.total_tracks  1382 non-null   float64       
 12  Cantidad artistas   1382 non-null   int64         
 13  artist_name         1382 non-null   object      

### Como el archivo va a cambiar mucho dependiendo de cuando hagamos la descarga, este notebook llegará hasta aquí donde se descargara el excel para uso como base en el archivo final

In [197]:
nombre_archivo = f"{fecha_corta}.xlsx"

if os.path.exists(nombre_archivo):
    # Obtener el tamaño actual del archivo existente
    tamaño_archivo_existente = os.path.getsize(nombre_archivo)
    
    # Guardar el DataFrame temporalmente en un archivo nuevo para comparar tamaños
    nombre_archivo_temporal = f"temp_{nombre_archivo}"
    datos.to_excel(nombre_archivo_temporal, index=False)
    
    # Obtener el tamaño del archivo temporal
    tamaño_archivo_nuevo = os.path.getsize(nombre_archivo_temporal)
    
    # Comparar los tamaños de los archivos
    if tamaño_archivo_existente == tamaño_archivo_nuevo:
        print(f"El archivo {nombre_archivo} ya existe y tiene el mismo tamaño. No se reemplazará.")
        # Eliminar el archivo temporal
        os.remove(nombre_archivo_temporal)
    else:
        print(f"El archivo {nombre_archivo} será sobrescrito.")
        # Sobrescribir el archivo original con el archivo temporal
        os.replace(nombre_archivo_temporal, nombre_archivo)
else:
    # Si el archivo no existe, crearlo
    final_tracks.to_excel(nombre_archivo, index=False)
    print(f"Archivo exportado como: {nombre_archivo}")

El archivo 2024-09-13.xlsx será sobrescrito.
